In [61]:
def lifeguard(file_number):
    import pandas as pd
    import numpy as np


    with open("%s.in" %file_number) as f:
        first_line = f.readline()
    N = int(first_line)

    print("N is: "+str(N))

    df = pd.read_csv("%s.in" %file_number, skiprows=1 ,header=None)
    "%s.out" %file_number
    #df = pd.read_csv('1.in', skiprows=1 ,header=None)
    df.columns = ["a"]
    df = df["a"].str.split(" ", n = 1, expand = True) 
    df.columns = ["a", "b"]
    df['a'] = df['a'].astype(int)
    df['b'] = df['b'].astype(int)
    df['min'] = df["a"]
    df['max'] = df["b"]
    df['diff'] = df['max'].astype(int) - df['min'].astype(int)
    df['segMin'] = '0'
    df['segMax'] = '0'

    df = df.sort_values("a")
    #print(df)

    stop = 0

    #get value from cell
    def getN(index, column):
        return int(df.iloc[index, df.columns.get_loc(column)])

    #update value of cell
    def update(index1, max, index2, a):
        df.iloc[index1, df.columns.get_loc(max)] = int(df.iloc[index2, df.columns.get_loc(a)])

    #print whole line with index i
    def printLine(i):
        print(getN(i,'a'),getN(i,'b'),getN(i,'min'),getN(i,'max'))

    m = 0
    stopInner = 0

    #calculate the minimum impact for firing a worker
    for i in range(N):
        if(stop == 1):
            print("lifeguard found")
            break        
        stopInner = 0        
        for j in range(i+1, N):
            if(stop == 1 or stopInner == 1):       
                break
            if(getN(i,'b') >= getN(j,'b')):
                stop = 1
                print("fully cover")
                printLine(i)
                printLine(j)
            #overlap
            elif(getN(j,'a') >= getN(i,'b')):
                stopInner = 1
            elif(getN(i,'b') > getN(j,'a')):                
                if(getN(i,'max') > getN(j,'a')):
                    update(i,'max',j,'a')                    
                if(getN(j,'min') < getN(i,'b')):
                    update(j,'min',i,'b')                                   
                if(getN(i,'max') <= getN(i,'min') or getN(j,'max') < getN(j, 'min')):
                    stop = 1
                    print("fully covered by many users")
                    printLine(i)
                    printLine(j)
            else:
                   None


    def printSeg(i):
        print("printSeg: ", end = '')
        print(getN(i,'a'),getN(i,'b'),getN(i,'segMin'),getN(i,'segMax'))

    #update segMax
    def updateSegMaX(segStart, segMax, bMax):
        df.iloc[segStart, df.columns.get_loc(segMax)] = bMax
    #update segMax
    def updateSegMaX(segStart, segMax, bMax):
        df.iloc[segStart, df.columns.get_loc(segMax)] = bMax

    df['diff'] = df['max'].astype(int) - df['min'].astype(int)

    min_influence = int(df[['diff']].min())


    #sum the overall hours before fire one worker
    segStart = 0

    while(segStart < N):        
        update(segStart,'segMin', segStart, 'a')
        update(segStart,'segMax', segStart, 'b')        
        bMax = getN(segStart,'b')        
        for i in range(segStart,N):
            if(i >= (N-1)):
                break
            if(getN(i+1,'a') > bMax): #a of i+1 > max of b in the previous cells (segStart to i)
                break
            if(getN(i+1,'b') > bMax):
                bMax = getN(i+1,'b')        
        updateSegMaX(segStart,'segMax',bMax)        
        segStart = i+1

    df['segDiff'] = df['segMax'].astype(int) - df['segMin'].astype(int)

    #df.to_csv("%s.csv" %file_number, index = False)

    output_initial = df['segDiff'].sum()
    print(output_initial)

    output = 0

    if(stop == 1):
        output = output_initial
    else:
        output = output_initial - min_influence

    print("stop:" + str(stop))
    print("output: " + str(output))


    with open("%s.out" %file_number, "w") as f:
        f.write(str(output))
    f.close()



In [65]:
lifeguard(10)

N is: 100000
74902061
stop:0
output: 74901843
